In [1]:
import os
import pandas as pd
from linkedin_scraper import extract_connections, extract_about, extract_experiences, scrape_profile, get_profile_links
from linkedin_nav import login_linkedin, go_to_connections, new_tab_url, close_tab, new_tab_link, scroll_with_spacebar

In [2]:
driver = login_linkedin()

In [3]:
go_to_connections(driver)


In [ ]:
scroll_with_spacebar(driver, n=100)

In [1]:
connections_df = extract_connections(driver)

NameError: name 'extract_connections' is not defined

In [46]:
# Split Name
connections_df["First Name"] = connections_df["Name"].str.split().str[0]
connections_df["Name Flag"] = connections_df["First Name"].str.contains(r"[^a-zA-Z]", regex=True).map({True: "!!!", False: ""})
# connections_df.rename(columns={'Title': 'Header'}, inplace=True)
connections_df = connections_df[["First Name", "Header", "Connected At", "URL", "Name", "Name Flag"]]

In [47]:
connections_df.to_csv("data/connections.csv", index=False, encoding="utf-8-sig")

In [55]:
connections_df = pd.read_csv("data/connections.csv")
experiences_df = pd.read_csv("data/experiences.csv")
profiles_df = pd.read_csv("data/profiles.csv")
people_df = connections_df.merge(profiles_df, on="URL", how="left")
people_df = people_df[["First Name", "Title", "Company", "Duration", "Connected At", "Header", "About", "URL", "Name", "Name Flag"]]
people_df.to_csv("data/people.csv", index=False, encoding="utf-8-sig")

In [ ]:
# Get list of URLs we've already scraped (to avoid duplicates)
# Load existing CSV files
if os.path.exists("data/profiles.csv"):
    profiles_df = pd.read_csv("data/profiles.csv")
    existing_urls = set(profiles_df["URL"].tolist())
else:
    # profiles_df = pd.DataFrame()
    existing_urls = []
if os.path.exists("data/experiences.csv"):
    experiences_df = pd.read_csv("data/experiences.csv")
else:
    experiences_df = pd.DataFrame()

profile_links, profile_urls = get_profile_links(driver)

# if url in existing_urls remove that link
new_profile_links = []

# Only keep what's not already in existing_urls
for link, url in zip(profile_links, profile_urls):
    if url not in existing_urls:
        new_profile_links.append(link)

In [52]:
max_profiles = 50
# Scrape only the new profile links
profiles_data = []
experiences_data = []

for i, link in enumerate(new_profile_links[:max_profiles]):
    try:        
        new_tab_link(driver, link)
        # Scrape profile
        profile_info = scrape_profile(driver)
        experiences = extract_experiences(driver)
        # Add to data lists
        profiles_data.append(profile_info)
        experiences_data.extend(experiences)
        
        # Close tab and return to main window
        close_tab(driver)
        
    except Exception as e:
        print(f"Error processing profile {i+1}: {e}")
        # Try to recover
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

# Update profiles
profiles_df = pd.concat([profiles_df, pd.DataFrame(profiles_data)], ignore_index=True)
profiles_df.drop_duplicates(subset=["URL"], keep="last", inplace=True)
profiles_df.to_csv("data/profiles.csv", index=False, encoding="utf-8-sig")

# Update experiences
experiences_df = pd.concat([experiences_df, pd.DataFrame(experiences_data)], ignore_index=True)
experiences_df.drop_duplicates(subset=["URL", "Company", "Title"], keep="last", inplace=True)
experiences_df.to_csv("data/experiences.csv", index=False, encoding="utf-8-sig")

View results

In [8]:
# Find problem profiles
# problem_profiles = profiles_df[pd.isna(profiles_df["Company"])]
# problem_profiles

In [ ]:
profiles_df

Scrape 1 profile URL

In [ ]:
new_tab_url(driver, "https://www.linkedin.com/in/khokharwaqas/")

In [ ]:
extract_experiences(driver, first_only=True)


In [ ]:
extract_about(driver)


In [ ]:
close_tab(driver)

In [ ]:
# import pandas as pd
# pd.set_option("display.max_rows", None)
# pd.reset_option("display.max_rows")